In [1]:
import numpy as np
import keras
#from keras.layers import Input, Dense, Conv2D, MaxPooling2D, concatenate, Flatten, AveragePooling2D
from keras.layers import*
from keras.models import Model
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import*
from keras.regularizers import l2

Using Theano backend.


In [2]:
Xtest = np.load("Xtest.npy")
Ytest = np.load("Ytest.npy")

In [3]:
print Xtest.shape
print Ytest.shape

(35880, 40, 40, 1)
(35880, 7)


In [4]:
XtestCenter = np.empty((Xtest.shape[0]/10,Xtest.shape[1],Xtest.shape[2],1))
YtestCenter = np.empty((Ytest.shape[0]/10,Ytest.shape[1]))

In [5]:
for i in range(0,XtestCenter.shape[0]):
    XtestCenter[i] = Xtest[10*i + 4]
    YtestCenter[i] = Ytest[10*i + 4]

In [6]:
print XtestCenter.shape
print YtestCenter.shape

(3588, 40, 40, 1)
(3588, 7)


In [7]:
img_input = Input(shape=(40,40,1), name='img_input')
x = Conv2D(64,7, strides=(2,2), activation = 'relu',padding='same')(img_input)
x = MaxPooling2D(pool_size=(3, 3), strides=(2,2),padding='same')(x)   
x = Conv2D(192,3,strides=(2,2),padding='same')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2,2),padding='same')(x)
x = BatchNormalization()(x)
#first inception layer 3a
x1 = Conv2D(64,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #1*1
x2 = Conv2D(96,3, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #3*3 reduce
x2 = Conv2D(128,3, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x2) #3*3 
x3 = Conv2D(16,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #5*5 reduce
x3 = Conv2D(32,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x3) #5*5 
x4 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(x) #pool proj
x4 = Conv2D(32,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x4) #1*1
x = concatenate([x1,x2,x3,x4], axis = 3)
x = BatchNormalization()(x)
#second inception layer 3b
x1 = Conv2D(128,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #1*1
x2 = Conv2D(128,3,padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #3*3 reduce
x2 = Conv2D(192,3, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x2) #3*3 
x3 = Conv2D(32,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #5*5 reduce
x3 = Conv2D(96,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x3) #5*5 
x4 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(x) #pool proj
x4 = Conv2D(64,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x4)
x = concatenate([x1,x2,x3,x4], axis = 3)
x = MaxPooling2D(pool_size=(3, 3), strides=(2,2),padding='same')(x)
x = BatchNormalization()(x)
#third inception layer 4a
x1 = Conv2D(192,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #1*1
x2 = Conv2D(96,3,padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #3*3 reduce
x2 = Conv2D(208,3, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x2) #3*3 
x3 = Conv2D(16,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #5*5 reduce
x3 = Conv2D(48,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x3) #5*5 
x4 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(x) #pool proj
x4 = Conv2D(64,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x4) #1*1
x = concatenate([x1,x2,x3,x4], axis = 3)
x = BatchNormalization()(x)
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(4096, activation='relu', W_regularizer=l2(0.0002))(x)
x = Dense(1024, activation='relu', W_regularizer=l2(0.0002))(x)
x = Dense(7, activation='softmax', W_regularizer=l2(0.0002))(x)
final_model = Model(inputs = [img_input], outputs = [x])
final_model.summary()

/home/amitoj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 1, padding="same", activation="relu", kernel_regularizer=<keras.reg...)`
  # Remove the CWD from sys.path while we load stuff.
/home/amitoj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, 3, padding="same", activation="relu", kernel_regularizer=<keras.reg...)`
  # This is added back by InteractiveShellApp.init_path()
/home/amitoj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="same", activation="relu", kernel_regularizer=<keras.reg...)`
  if sys.path[0] == '':
/home/amitoj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, 5, padding="same", activation="relu", kernel_regular

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_input (InputLayer)          (None, 40, 40, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 20, 64)   3200        img_input[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 10, 10, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 5, 5, 192)    110784      max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

dense_3 (Dense)                 (None, 7)            7175        dense_2[0][0]                    
Total params: 8,584,999
Trainable params: 8,582,119
Non-trainable params: 2,880
__________________________________________________________________________________________________


In [8]:
sgd = SGD(lr=0.01, nesterov=False)
final_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [9]:
final_model.load_weights("batchnorm-0.2dropout-retry-31-0.5765.hdf5")

In [10]:
preds = final_model.predict(XtestCenter)

In [11]:
print preds[78]

[  1.25793412e-01   1.58136021e-02   7.60588199e-02   7.81715512e-01
   1.22019861e-04   4.73836757e-04   2.27798719e-05]


In [12]:
Y_pred = np.argmax(preds, axis = 1)
Y_test = np.argmax(YtestCenter, axis = 1)

In [13]:
ans = 0
for i in range(Y_test.shape[0]):
    if Y_test[i] == Y_pred[i]:
        #print Y_test[i], Y_pred[i]
        ans = ans + 1    
print "Test Accuracy is"
print(ans/float(Y_test.shape[0]))*100 

Test Accuracy is
61.4548494983


In [14]:
Y_pred_top2 = preds.argsort(axis = 1)[:,5:7]
print Y_pred_top2.shape

(3588, 2)


In [15]:
ans = 0
for i in range(Y_test.shape[0]):
    if Y_test[i] == Y_pred_top2[i][0] or Y_test[i] == Y_pred_top2[i][1]:
        #print Y_test[i], Y_pred[i]
        ans = ans + 1    
print "Top 2 Accuracy is"
print(ans/float(Y_test.shape[0]))*100

Top 2 Accuracy is
77.8428093645


In [16]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [17]:
expr = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}
Y_expr = np.array([expr[a] for a in Y_test])
Y_pred_expr = np.array([expr[a] for a in Y_pred])

In [19]:
plt.rcParams.update({'font.size': 8})
class_names =[]
for k in range(7):
    class_names.append(expr[k])
cnf_matrix = confusion_matrix(Y_expr, Y_pred_expr)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

plt.savefig('BNconf.png')
# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
plt.savefig('BNnConf.png')